<pre>
1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>

2. Code the model to classify data like below image

<img src='https://i.imgur.com/33ptOFy.png'>

3. Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.

4. Save your model at every epoch if your validation accuracy is improved from previous epoch. 

5. you have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
6. If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

7. You have to stop the training if your validation accuracy is not increased in last 2 epochs.

8. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

9. use cross entropy as loss function

10. Try the architecture params as given below. 
</pre>

<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# !ls '/content/gdrive/My Drive/Colab Notebooks/Callbacks'

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
from sklearn.metrics import roc_auc_score,f1_score
import random as rn
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.python.keras import backend as K
import datetime
%load_ext tensorboard


In [2]:
tf.__version__

'2.1.0'

In [5]:
!rm -rf ./logs/

In [6]:
data = pd.read_csv('data.csv')

In [7]:
Y = data['label']
X = data.drop(['label'],axis=1)
print(X.shape,Y.shape)

(20000, 2) (20000,)


In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=42)
y_train = tf.keras.utils.to_categorical(y_train, 2) 
y_test = tf.keras.utils.to_categorical(y_test, 2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(14000, 2)
(14000, 2)
(6000, 2)
(6000, 2)


In [9]:
class MyClass(tf.keras.callbacks.Callback):
    def __init__(self,validation_data):
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        ## on begin of training, we are creating a instance varible called history
        ## it is a dict with keys [loss, acc, F1 score, auc_score]
        self.history={'loss': [],'acc': [],'val_loss': [],'val_acc': [],'F1_score':[],'auc_score':[]}
        
    def on_epoch_end(self, epoch, logs={}):
        ## on end of each epoch, we will get logs and update the self.history dict
        self.history['loss'].append(logs.get('loss'))
        self.history['acc'].append(logs.get('acc'))
        if logs.get('val_loss', -1) != -1:
            self.history['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_acc', -1) != -1:
            self.history['val_acc'].append(logs.get('val_acc'))

        val_predict = (np.asarray(self.model.predict(self.x_val))).round()
        val_targ = self.y_val
        f1 = round(f1_score(val_targ, val_predict,average='micro'),4)
        auc = round(roc_auc_score(val_targ,val_predict),4)
        print(' - f1 score - {} - auc_score - {}'.format(f1,auc))
        self.history['F1_score'].append(logs.get('F1_score'))
        self.history['auc_score'].append(logs.get('auc_score'))

        
            
           

<h1>Model  1</h1>



In [10]:
class LrReducer(tf.keras.callbacks.Callback):
    def __init__(self):
        super(tf.keras.callbacks.Callback, self).__init__()
        self.wait = 0
        self.best_score = -1.

    def on_epoch_end(self, epoch, logs=None):
        current_score = logs.get('val_acc')
        self.wait = 0
        # lr = self.get_value(self.model.optimizer.lr)
        lr  = K.get_value(self.model.optimizer.lr)
        print('Current Learning rate is ',lr)
        if current_score is not None and current_score > self.best_score:
            self.best_score = current_score
            K.set_value(self.model.optimizer.lr, (lr - (lr*0.1)))
        else:
            if self.wait % 3 == 0:
                K.set_value(self.model.optimizer.lr, (lr - (lr*0.05)))
        self.wait += 1


In [11]:
class TerminateNaN(tf.keras.callbacks.Callback):
        
    def on_epoch_end(self, epoch, logs={}):
        loss = logs.get('loss')
        if loss is not None:
            if np.isnan(loss) or np.isinf(loss):
                print("Invalid loss and terminated at epoch {}".format(epoch))
                self.model.stop_training = True

In [12]:
!rm -rf ./logs/

In [13]:
#Input layer
input_layer = Input(shape=(2,))

#Dense hidden layer 1
layer1 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform( minval=0.,maxval = 1.,seed=30))(input_layer)

#Dense hidden layer 2
layer2 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0.,maxval = 1.,seed=42))(layer1)

#Dense hidden layer 3
layer3 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0.,maxval = 1.,seed=22))(layer2)

#Dense hidden layer 4
layer4 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0.,maxval = 1.,seed=32))(layer3)

#Dense hidden layer 5
layer5 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0.,maxval = 1.,seed=12))(layer4)

#output layer
output = Dense(2,activation='softmax',kernel_initializer=tf.keras.initializers.RandomUniform(minval = 0.,maxval = 1.,seed=0))(layer5)

#Creating a model
model = Model(inputs=input_layer,outputs=output)


#Callbacks
history_own = MyClass(validation_data=(X_test,y_test)) 
earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=2, verbose=1)
lrschedule = LrReducer()
filepath="model_save/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='max')
terminate_nan = TerminateNaN()

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)



optimizer = tf.keras.optimizers.SGD(0.01)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train,y_train,epochs=10, validation_data=(X_test,y_test), batch_size=30, callbacks=[history_own,earlystop,checkpoint,terminate_nan,lrschedule,tensorboard_callback])


Train on 14000 samples, validate on 6000 samples
Epoch 1/10
13860/14000 [============================>.] - ETA: 0s - loss: 0.6971 - accuracy: 0.4987 - f1 score - 0.4968 - auc_score - 0.5

Epoch 00001: val_accuracy improved from -inf to 0.49683, saving model to model_save/weights-01-0.4968.hdf5
Current Learning rate is  0.01
14000/14000 [==============================] - 9s 656us/sample - loss: 0.6971 - accuracy: 0.4986 - val_loss: 0.6961 - val_accuracy: 0.4968
Epoch 2/10
13890/14000 [============================>.] - ETA: 0s - loss: 0.6956 - accuracy: 0.5001 - f1 score - 0.4968 - auc_score - 0.5

Epoch 00002: val_accuracy did not improve from 0.49683
Current Learning rate is  0.0095
14000/14000 [==============================] - 6s 455us/sample - loss: 0.6955 - accuracy: 0.5005 - val_loss: 0.6948 - val_accuracy: 0.4968
Epoch 3/10
13890/14000 [============================>.] - ETA: 0s - loss: 0.6957 - accuracy: 0.4962 - f1 score - 0.4968 - auc_score - 0.5

Epoch 00003: val_accuracy did 

In [57]:
!kill 11529

In [14]:
%tensorboard --logdir logs/fit

In [68]:
!rm -rf ./logs/

# Model 2

In [91]:
#Input layer
input_layer = Input(shape=(2,))

#Dense hidden layer 1
layer1 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval = 0.,maxval=1.,seed=30))(input_layer)

#Dense hidden layer 2
layer2 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0.,maxval = 1.,seed=42))(layer1)

#Dense hidden layer 3
layer3 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0.,maxval = 1.,seed=22))(layer2)

#Dense hidden layer 4
layer4 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0.,maxval = 1.,seed=32))(layer3)

#Dense hidden layer 5
layer5 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0.,maxval = 1.,seed=12))(layer4)

#output layer
output = Dense(2,activation='softmax',kernel_initializer=tf.keras.initializers.RandomUniform(minval = 0.,maxval = 1.,seed=0))(layer5)

#Creating a model
model = Model(inputs=input_layer,outputs=output)


#Callbacks
history_own = MyClass(validation_data=(X_test,y_test)) 
earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=2, verbose=1)
lrschedule = LrReducer()
filepath="model_save/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='max')
terminate_nan = TerminateNaN()

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True)



optimizer = tf.keras.optimizers.SGD(0.01)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train,y_train,epochs=10, validation_data=(X_test,y_test), batch_size=30, callbacks=[history_own,earlystop,checkpoint,terminate_nan,lrschedule,tensorboard_callback])


Train on 14000 samples, validate on 6000 samples
Epoch 1/10
13860/14000 [============================>.] - ETA: 0s - loss: 2076603827.9707 - accuracy: 0.4981 - f1 score - 0.4968 - auc_score - 0.5

Epoch 00001: val_accuracy improved from -inf to 0.49683, saving model to model_save/weights-01-0.4968.hdf5
Current Learning rate is  0.01
14000/14000 [==============================] - 9s 677us/sample - loss: 2055837789.6979 - accuracy: 0.4979 - val_loss: 0.6933 - val_accuracy: 0.4968
Epoch 2/10
13950/14000 [============================>.] - ETA: 0s - loss: 0.6932 - accuracy: 0.4993 - f1 score - 0.5032 - auc_score - 0.5

Epoch 00002: val_accuracy improved from 0.49683 to 0.50317, saving model to model_save/weights-02-0.5032.hdf5
Current Learning rate is  0.0095
14000/14000 [==============================] - 7s 533us/sample - loss: 0.6932 - accuracy: 0.4994 - val_loss: 0.6931 - val_accuracy: 0.5032
Epoch 3/10
13950/14000 [============================>.] - ETA: 0s - loss: 0.6932 - accuracy: 0.5

In [90]:
!rm -rf ./logs/

In [93]:
!kill 13018

In [94]:
%tensorboard --logdir logs/fit

# Model 3

In [147]:
pip install keras --upgrade

     |████████████████████████████████| 377 kB 120 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [146]:
#Input layer
input_layer = Input(shape=(2,))

#Dense hidden layer 1
layer1 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(seed=30))(input_layer)

#Dense hidden layer 2
layer2 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(seed=42))(layer1)

#Dense hidden layer 3
layer3 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(seed=22))(layer2)

#Dense hidden layer 4
layer4 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(seed=32))(layer3)

#Dense hidden layer 5
layer5 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(seed=12))(layer4)

#output layer
output = Dense(2,activation='softmax',kernel_initializer=tf.keras.initializers.he_uniform(seed=0))(layer5)

#Creating a model
model = Model(inputs=input_layer,outputs=output)


#Callbacks
history_own = MyClass(validation_data=(X_test,y_test)) 
earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=2, verbose=1)
lrschedule = LrReducer()
filepath="model_save/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='max')
terminate_nan = TerminateNaN()

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=10, write_graph=True,write_grads=True)



optimizer = tf.keras.optimizers.SGD(0.01)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train,y_train,epochs=10, validation_data=(X_test,y_test), batch_size=30, callbacks=[history_own,earlystop,checkpoint,terminate_nan,lrschedule,tensorboard_callback])


Train on 14000 samples, validate on 6000 samples
Epoch 1/10
13950/14000 [============================>.] - ETA: 0s - loss: 0.6647 - accuracy: 0.5933 - f1 score - 0.6505 - auc_score - 0.651

Epoch 00001: val_accuracy improved from -inf to 0.65050, saving model to model_save/weights-01-0.6505.hdf5
Current Learning rate is  0.01
14000/14000 [==============================] - 10s 694us/sample - loss: 0.6643 - accuracy: 0.5939 - val_loss: 0.6364 - val_accuracy: 0.6505
Epoch 2/10
13980/14000 [============================>.] - ETA: 0s - loss: 0.6196 - accuracy: 0.6574 - f1 score - 0.6405 - auc_score - 0.6394

Epoch 00002: val_accuracy did not improve from 0.65050
Current Learning rate is  0.0095
14000/14000 [==============================] - 7s 523us/sample - loss: 0.6194 - accuracy: 0.6576 - val_loss: 0.6406 - val_accuracy: 0.6405
Epoch 3/10
13890/14000 [============================>.] - ETA: 0s - loss: 0.6093 - accuracy: 0.6643 - f1 score - 0.6582 - auc_score - 0.6588

Epoch 00003: val_accu

In [24]:
!kill 10595

In [25]:
%tensorboard --logdir logs/fit

# Model 4

In [141]:
!rm -rf ./logs/

In [142]:
#Input layer
input_layer = Input(shape=(2,))

#Dense hidden layer 1
layer1 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(seed=30))(input_layer)

#Dense hidden layer 2
layer2 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(seed=42))(layer1)

#Dense hidden layer 3
layer3 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(seed=22))(layer2)

#Dense hidden layer 4
layer4 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(seed=32))(layer3)

#Dense hidden layer 5
layer5 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(seed=12))(layer4)

#output layer
output = Dense(2,activation='softmax',kernel_initializer=tf.keras.initializers.he_uniform(seed=0))(layer5)

#Creating a model
model = Model(inputs=input_layer,outputs=output)


#Callbacks
history_own = MyClass(validation_data=(X_test,y_test)) 
earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=2, verbose=1)
lrschedule = LrReducer()
filepath="model_save/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='max')
terminate_nan = TerminateNaN()

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True)



optimizer = tf.keras.optimizers.Adagrad(0.01)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train,y_train,epochs=10, validation_data=(X_test,y_test), batch_size=30, callbacks=[history_own,earlystop,checkpoint,terminate_nan,lrschedule,tensorboard_callback])


Train on 14000 samples, validate on 6000 samples
Epoch 1/10
13890/14000 [============================>.] - ETA: 0s - loss: 0.6580 - accuracy: 0.6131 - f1 score - 0.6485 - auc_score - 0.648

Epoch 00001: val_accuracy improved from -inf to 0.64850, saving model to model_save/weights-01-0.6485.hdf5
Current Learning rate is  0.01
14000/14000 [==============================] - 9s 644us/sample - loss: 0.6577 - accuracy: 0.6136 - val_loss: 0.6291 - val_accuracy: 0.6485
Epoch 2/10
13950/14000 [============================>.] - ETA: 0s - loss: 0.6125 - accuracy: 0.6636 - f1 score - 0.6523 - auc_score - 0.6515

Epoch 00002: val_accuracy improved from 0.64850 to 0.65233, saving model to model_save/weights-02-0.6523.hdf5
Current Learning rate is  0.0095
14000/14000 [==============================] - 8s 576us/sample - loss: 0.6123 - accuracy: 0.6639 - val_loss: 0.6283 - val_accuracy: 0.6523
Epoch 3/10
13980/14000 [============================>.] - ETA: 0s - loss: 0.6035 - accuracy: 0.6692 - f1 scor

In [144]:
!kill 14240

In [145]:
%tensorboard --logdir logs/fit